# Tutorial <br>

Before starting, remember to activate the environment:  <br>
**source env/bin/activate**





In [ ]:
from intrinsic_dimension import intrinsic_dimension, section_id, secondary_structure_id
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import seaborn as sns
import logging
from moleculekit.molecule import Molecule

Protein: **N-Terminal domain of ribosomial protein L9 (NTL9)** <br>
Each trajectory contains 2000 frames (about 2 μs each) and one state: <br>
- Folded, trajectories 1, 3, 5
- Unfolded, trajectories 0, 2, 4   



<h4><p style=' color:MediumVioletRed;' >Section_id</h4>  

In [ ]:
proj = {'dihedrals':'psi'} #changed default angles to compute
states = [0, 1, 2, 3, 4, 5]
colors = plt.cm.jet(np.linspace(0, 1, len(states)))
all_results = []
for s in states: 
    if s == 0:
        results = section_id(topology='villin/2F4K.pdb', trajectory=f'villin/2F4K_{s}.xtc', window_size = 15, stride = 3, projection_method='Dihedrals', id_method='global' , projection_kwargs=proj, verbose=True)
    else: #slicing is the same for all trajectories
        results = section_id(topology='villin/2F4K.pdb', trajectory=f'villin/2F4K_{s}.xtc', window_size = 15, stride = 3, projection_method='Dihedrals', id_method='global' , projection_kwargs=proj, verbose=False)
    results["state"] = s  
    all_results.append(results)
results = pd.concat(all_results, ignore_index=True)
results['segment'] = results.apply(lambda row: f"{row['start']}-{row['end']}", axis=1) #define segment name
unique_segments = results['segment'].unique()
states = sorted(results['state'].unique())
plt.figure(figsize=(10, 5))

for state, color in zip(states, colors):
    subset = results[results['state'] == state]
    y = subset['entire simulation'].values
    x = [seg for seg in subset['segment']]
    plt.scatter(x, y, color=color, edgecolor='black', linewidth=0.8, s=100, label=f"{state}")
plt.xticks(rotation=45)
plt.xlabel("Window")
plt.ylabel("Global ID")
plt.title("Local ID of protein's sections")
plt.legend(title="Trajectory", bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.tight_layout()
plt.show()
